# Bonds #
## 26. More mischievous conventions ##

In [1]:
import QuantLib as ql
import pandas as pd

### The case of the two slightly different prices ###

In [2]:
today = ql.Date(27, ql.January, 2011)
ql.Settings.instance().evaluationDate = today

In [3]:
issueDate = ql.Date(28, ql.January, 2011)
maturity = ql.Date(31, ql.August, 2020)
schedule = ql.Schedule(issueDate, maturity, ql.Period(ql.Semiannual),
                       ql.UnitedStates(ql.UnitedStates.GovernmentBond),
                       ql.Unadjusted, ql.Unadjusted,
                       ql.DateGeneration.Backward, False)
bond = ql.FixedRateBond(1, 100.0, schedule, [0.03625],
                        ql.ActualActual(ql.ActualActual.Bond),
                        ql.Unadjusted, 100.0)

In [4]:
bond_yield = 0.034921
P1 = bond.dirtyPrice(bond_yield, bond.dayCounter(), ql.Compounded, ql.Semiannual)

In [5]:
flat_curve = ql.FlatForward(bond.settlementDate(), bond_yield,
                            ql.ActualActual(ql.ActualActual.Bond),
                            ql.Compounded, ql.Semiannual)
engine = ql.DiscountingBondEngine(ql.YieldTermStructureHandle(flat_curve))
bond.setPricingEngine(engine)
P2 = bond.dirtyPrice()

In [6]:
pd.DataFrame([(P1,P2)], columns=['with yield', 'with curve'], index=[''])

,with yield,with curve
,101.076816,101.079986


### What happened? ###

In [9]:
pd.DataFrame([(ql.as_coupon(c).accrualStartDate(), ql.as_coupon(c).accrualEndDate()) for c in bond.cashflows()[:-1]],
             columns = ('start date','end date'), index = range(1, len(bond.cashflows())))

,start date,end date
1,"January 28th, 2011","February 28th, 2011"
2,"February 28th, 2011","August 31st, 2011"
3,"August 31st, 2011","February 29th, 2012"
4,"February 29th, 2012","August 31st, 2012"
5,"August 31st, 2012","February 28th, 2013"
6,"February 28th, 2013","August 31st, 2013"
7,"August 31st, 2013","February 28th, 2014"
8,"February 28th, 2014","August 31st, 2014"
9,"August 31st, 2014","February 28th, 2015"
10,"February 28th, 2015","August 31st, 2015"


In [10]:
dayCounter = ql.ActualActual(ql.ActualActual.Bond)
T = dayCounter.yearFraction(ql.Date(28, ql.January, 2011), ql.Date(28, ql.February, 2011),
                            ql.Date(28,ql.August,2010), ql.Date(28,ql.February,2011))
print(T)

0.08423913043478261


In [12]:
print(dayCounter.yearFraction(ql.Date(28, ql.January, 2011), ql.Date(28, ql.February, 2011),
                              ql.Date(28,ql.February,2010), ql.Date(28,ql.February,2011)))

0.08493150684931507


In [13]:
y = ql.InterestRate(bond_yield, dayCounter, ql.Compounded, ql.Semiannual)
print(y.discountFactor(T))

0.997087920498809


### Yield-based calculation ###

In [15]:
data = []
for i, c in enumerate(bond.cashflows()[:-1]):
    c = ql.as_coupon(c)
    A = c.amount()
    T = c.accrualPeriod()
    D = y.discountFactor(T)
    D_cumulative = D if i == 0 else D * data[-1][3]
    A_discounted = A*D_cumulative
    data.append((A,T,D,D_cumulative,A_discounted))
data.append((100,'','',D_cumulative,100*D_cumulative))
data = pd.DataFrame(data,
                    columns = ('amount', 'T', 'discount', 'discount (cum.)', 'amount(disc.)'),
                    index = ['']*len(data))
data

,amount,T,discount,discount (cum.),amount(disc.)
,0.305367,0.084239,0.997088,0.997088,0.304478
,1.812500,0.5,0.982839,0.979977,1.776208
,1.812500,0.5,0.982839,0.963160,1.745727
,1.812500,0.5,0.982839,0.946631,1.715769
,1.812500,0.5,0.982839,0.930386,1.686325
,1.812500,0.5,0.982839,0.914420,1.657386
,1.812500,0.5,0.982839,0.898728,1.628944
,1.812500,0.5,0.982839,0.883305,1.600990
,1.812500,0.5,0.982839,0.868146,1.573515
,1.812500,0.5,0.982839,0.853248,1.546513


In [17]:
print(sum(data['amount(disc.)']))

101.07681646503603


In [18]:
print(bond.dirtyPrice(bond_yield, bond.dayCounter(), ql.Compounded, ql.Semiannual))

101.07681646503603


### Curve-based calculation ###

In [20]:
data = []
for c in bond.cashflows()[:-1]:
    A = c.amount()
    D = flat_curve.discount(c.date())
    A_discounted = A*D
    data.append((A,D,A_discounted))
data.append((100.0,D,100.0*D))
data = pd.DataFrame(data,
                    columns = ('amount', 'discount', 'amount (disc.)'),
                    index = ['']*len(data))
data

,amount,discount,amount (disc.)
,0.305367,0.997119,0.304487
,1.812500,0.980008,1.776264
,1.812500,0.963190,1.745782
,1.812500,0.946661,1.715823
,1.812500,0.930415,1.686378
,1.812500,0.914449,1.657438
,1.812500,0.898756,1.628995
,1.812500,0.883332,1.601040
,1.812500,0.868174,1.573565
,1.812500,0.853275,1.546561


In [22]:
print(sum(data['amount (disc.)']))

101.0799861183387


In [23]:
print(bond.dirtyPrice())

101.0799861183387


In [25]:
flat_curve.discount(ql.Date(28, ql.February, 2011))

0.9971191880350325

In [26]:
print(dayCounter.yearFraction(ql.Date(28, ql.January, 2011), ql.Date(28, ql.February, 2011)))

0.08333333333333333


In [27]:
T = dayCounter.yearFraction(ql.Date(28, ql.January, 2011), ql.Date(28, ql.February, 2011),
                            ql.Date(28,ql.August,2010), ql.Date(28,ql.February,2011))
print(T)

0.08423913043478261


In [28]:
P_y = bond.dirtyPrice(bond_yield, bond.dayCounter(), ql.Compounded, ql.Semiannual)
D_y = y.discountFactor(T)
P_c = bond.dirtyPrice()
D_c = flat_curve.discount(ql.Date(28, ql.February, 2011))
print(P_y)
print(P_c*(D_y/D_c))

101.07681646503603
101.0768164650361
